<a href="https://colab.research.google.com/github/emiliomantilla/AIgent007/blob/main/Agentic_Community_Resource_Navigator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
import datetime
from typing import Dict, Any, List

# Import the google_search tool (simulated for this environment)
# In a real environment, you would import actual web scraping libraries or specific APIs.
# For this hackathon context, we'll use the provided google_search tool.
from google_search import search as google_search_tool

# --- Configuration ---
# Your Gemini API key. Leave as empty string for Canvas environment to provide.
API_KEY = ""
GEMINI_API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={API_KEY}"

# Supported resource types and their common search keywords
RESOURCE_TYPES = {
    "food bank": ["food bank", "food pantry", "meal program"],
    "shelter": ["homeless shelter", "emergency housing", "night shelter"],
    "free clinic": ["free clinic", "community health center", "low-cost medical help"],
    "job support center": ["job assistance", "employment services", "career guidance"],
    "community event": ["community events", "local happenings", "free activities"],
    "mental health service": ["mental health support", "counseling services", "therapy for homeless"],
    "legal aid": ["free legal aid", "housing legal assistance", "homeless legal services"]
}

# In-memory storage for user preferences and subscriptions (for hackathon demo)
# In a real application, this would be a persistent database (e.g., Firestore).
user_preferences: Dict[str, Any] = {
    "preferred_language": "English",
    "accessibility_needs": [], # e.g., ["wheelchair accessible", "sign language interpreter"]
    "subscribed_categories": [] # e.g., ["food bank", "community event"]
}

# --- Helper Functions ---

async def call_gemini_api(prompt: str, schema: Dict = None) -> Any:
    """
    Calls the Gemini API with the given prompt and optional response schema.
    """
    chat_history = []
    chat_history.push({ "role": "user", "parts": [{ "text": prompt }] })

    payload = {
        "contents": chat_history,
    }
    if schema:
        payload["generationConfig"] = {
            "responseMimeType": "application/json",
            "responseSchema": schema
        }

    try:
        response = await fetch(GEMINI_API_URL, {
            "method": "POST",
            "headers": { "Content-Type": "application/json" },
            "body": json.dumps(payload)
        })
        result = await response.json()

        if result.get("candidates") and result["candidates"][0].get("content") and result["candidates"][0]["content"].get("parts"):
            text_content = result["candidates"][0]["content"]["parts"][0]["text"]
            if schema:
                return json.loads(text_content) # Parse JSON if schema was used
            return text_content
        else:
            print(f"Error: Unexpected Gemini API response structure: {result}")
            return None
    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        return None

async def search_resources_simulated(query_params: Dict[str, Any]) -> List[Dict[str, str]]:
    """
    Simulates searching for resources using the google_search tool.
    In a real application, this would involve dedicated web scraping or database queries.
    """
    resource_type = query_params.get("resource_type", "resource").lower()
    location = query_params.get("location", "London").capitalize() # Default to London
    time_context = query_params.get("time_context", "today")
    keywords = RESOURCE_TYPES.get(resource_type, [resource_type])

    search_query = f"{' OR '.join(keywords)} {location} {time_context}"
    if query_params.get("open_on_weekends"):
        search_query += " open on weekends"
    if query_params.get("accessibility_needs"):
        search_query += f" {', '.join(query_params['accessibility_needs'])}"

    print(f"\n[Agent Action] Searching Google for: '{search_query}'")
    try:
        search_results = google_search_tool(queries=[search_query])
        results = []
        if search_results and search_results[0].results:
            for item in search_results[0].results[:3]: # Limit to top 3 results for brevity
                results.append({
                    "title": item.source_title or "Unknown Source",
                    "snippet": item.snippet or "No description available.",
                    "url": item.url or "#"
                })
        return results
    except Exception as e:
        print(f"Error during simulated resource search: {e}")
        return []

# --- Agent Core Logic ---

async def process_user_request(user_input: str) -> str:
    """
    Processes a high-level user request using agentic principles.
    """
    print(f"\n[User] {user_input}")

    # Step 1: Understand user intent and extract key entities using Gemini
    # Define a schema for structured output from Gemini
    intent_schema = {
        "type": "OBJECT",
        "properties": {
            "intent": {"type": "STRING", "description": "The user's primary intent (e.g., 'find_resource', 'set_preference', 'subscribe', 'give_feedback')."},
            "resource_type": {"type": "STRING", "description": "Type of resource requested (e.g., 'food bank', 'shelter', 'mental health service'). Null if not a resource request."},
            "location": {"type": "STRING", "description": "Geographic location mentioned in the request. Null if not specified."},
            "time_context": {"type": "STRING", "description": "Time context (e.g., 'today', 'tonight', 'this weekend', 'next week'). Null if not specified."},
            "open_on_weekends": {"type": "BOOLEAN", "description": "True if user explicitly asks for weekend availability."},
            "preferences_to_set": {
                "type": "OBJECT",
                "properties": {
                    "preferred_language": {"type": "STRING"},
                    "accessibility_needs": {"type": "ARRAY", "items": {"type": "STRING"}}
                },
                "additionalProperties": False
            },
            "category_to_subscribe": {"type": "STRING", "description": "The resource category user wants to subscribe to."},
            "feedback_text": {"type": "STRING", "description": "The text of the user's feedback."},
            "urgency": {"type": "STRING", "description": "Inferred urgency (e.g., 'high', 'medium', 'low')."}
        },
        "propertyOrdering": ["intent", "resource_type", "location", "time_context", "open_on_weekends", "preferences_to_set", "category_to_subscribe", "feedback_text", "urgency"]
    }

    prompt_for_intent = f"""
    Analyze the following user request and extract the intent and relevant parameters as a JSON object.
    Consider the current user preferences: {json.dumps(user_preferences)}.
    Current date and time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}.

    User request: "{user_input}"

    If the user asks for a resource, identify the 'resource_type', 'location', 'time_context', and 'open_on_weekends'.
    If the user wants to set preferences, identify 'preferences_to_set'.
    If the user wants to subscribe, identify 'category_to_subscribe'.
    If the user provides feedback, identify 'feedback_text'.
    Infer 'urgency' based on keywords like 'urgent', 'immediately', 'need now'.
    """
    parsed_request = await call_gemini_api(prompt_for_intent, intent_schema)

    if not parsed_request:
        return "I'm sorry, I couldn't understand your request. Could you please rephrase it?"

    intent = parsed_request.get("intent")
    print(f"[Agent Thought] Parsed Intent: {intent}, Params: {parsed_request}")

    response_message = ""

    if intent == "find_resource":
        resource_type = parsed_request.get("resource_type")
        location = parsed_request.get("location")
        time_context = parsed_request.get("time_context")
        open_on_weekends = parsed_request.get("open_on_weekends", False)
        urgency = parsed_request.get("urgency", "low")

        if not resource_type:
            return "Please specify what kind of resource you are looking for (e.g., food bank, shelter, clinic)."

        # Incorporate user preferences into the search query
        effective_accessibility_needs = user_preferences["accessibility_needs"] + parsed_request.get("accessibility_needs", [])

        query_params = {
            "resource_type": resource_type,
            "location": location,
            "time_context": time_context,
            "open_on_weekends": open_on_weekends,
            "accessibility_needs": effective_accessibility_needs,
            "urgency": urgency # For future prioritization logic
        }

        # Step 2: Execute queries across sources (simulated)
        found_resources = await search_resources_simulated(query_params)

        # Step 3: Reason about credibility and gaps (simulated/LLM-driven)
        # In a real system, you'd have more sophisticated logic here.
        # For a hackathon, we can use the LLM to frame the results.
        if found_resources:
            resource_list_str = "\n".join([
                f"- **{res['title']}**: {res['snippet']} [More Info: {res['url']}]"
                for res in found_resources
            ])
            # Use Gemini to format the response, adding a note about verification
            response_prompt = f"""
            The user requested a '{resource_type}' in '{location}' for '{time_context}'.
            I found the following resources:
            {resource_list_str}

            Please format a helpful, empathetic response for a homeless person.
            Include a strong recommendation to call ahead to verify details and availability,
            and mention that information can change quickly.
            Also, remind them of their set preferences like preferred language and accessibility needs
            if they were factored into the search.
            """
            response_message = await call_gemini_api(response_prompt)
        else:
            response_message = f"I couldn't find any '{resource_type}' resources matching your criteria in '{location}' for '{time_context}'. Information for these services can be limited or change frequently. Please try a different query or location, or consider contacting local community centers directly."

    elif intent == "set_preference":
        prefs_to_set = parsed_request.get("preferences_to_set", {})
        if prefs_to_set:
            for key, value in prefs_to_set.items():
                if key in user_preferences:
                    if isinstance(user_preferences[key], list) and isinstance(value, list):
                        user_preferences[key] = list(set(user_preferences[key] + value)) # Merge lists
                    else:
                        user_preferences[key] = value
            response_message = f"Your preferences have been updated. Current preferences: {user_preferences}"
        else:
            response_message = "I didn't detect any preferences to set. What would you like to change?"

    elif intent == "subscribe":
        category = parsed_request.get("category_to_subscribe")
        if category and category in RESOURCE_TYPES:
            if category not in user_preferences["subscribed_categories"]:
                user_preferences["subscribed_categories"].append(category)
                response_message = f"You are now subscribed to alerts for new '{category}' resources and updates. I will proactively notify you if new information becomes available (in a real system, this would be via email/SMS)."
            else:
                response_message = f"You are already subscribed to '{category}' alerts."
        else:
            response_message = "I can only subscribe you to known resource categories like food banks, shelters, etc. Which category would you like to subscribe to?"

    elif intent == "give_feedback":
        feedback = parsed_request.get("feedback_text")
        if feedback:
            # In a real system, this feedback would be logged to a database for analysis
            print(f"\n[Agent Log] User Feedback Received: '{feedback}'")
            response_message = "Thank you for your feedback! This helps us improve the service."
        else:
            response_message = "Please provide your feedback text."

    else:
        response_message = "I understand you're looking for assistance. Please tell me what kind of help you need, your location, and any specific requirements."

    return response_message

# --- Main Interaction Loop ---

async def main():
    print("Welcome to the Community Resource Navigator Agent!")
    print("I can help you find food banks, shelters, clinics, job support, community events, and mental health services.")
    print("You can also set preferences (e.g., 'My preferred language is Spanish'), subscribe to updates (e.g., 'Subscribe to food bank alerts'), or give feedback.")
    print("Type 'exit' to quit.")

    # Initialize user preferences with defaults or load from a simulated profile
    # For a demo, we'll just use the initial `user_preferences` dict.

    # Simulate proactive alert (for subscribed categories)
    # In a real system, this would be triggered by a background process monitoring data changes.
    if user_preferences["subscribed_categories"]:
        print("\n[Proactive Alert Simulation]")
        for category in user_preferences["subscribed_categories"]:
            print(f"Checking for new '{category}' updates...")
            # Simulate finding new info or changes
            if datetime.datetime.now().minute % 5 == 0: # A simple, arbitrary trigger for demo
                print(f"  > Simulated new information found for '{category}'! (e.g., 'New pop-up food pantry at XYZ location on Saturday.')")
        print("[End Proactive Alert Simulation]\n")


    while True:
        user_input = input("\nHow can I help you today? ").strip()
        if user_input.lower() == 'exit':
            print("Thank you for using the Community Resource Navigator. Goodbye!")
            break

        response = await process_user_request(user_input)
        print(f"\n[Agent] {response}")
        print(f"\n[Agent Info] Current User Preferences: {user_preferences}") # Show preferences for demo

# To run this in a local environment, you'd typically use `asyncio.run(main())`
# In the Canvas environment, the `await` keyword can be used directly in the global scope
# or within an async function called from the global scope.
# For this script to be runnable in the Canvas, we'll define `main` as an async function
# and assume the environment handles its execution.